In [ ]:
#| default_exp config

In [ ]:
#| export
from __future__ import annotations

## Config

Simple namespace to store configuration with a context manager to make temporary changes.

# Prologue

In [ ]:
#| export
import inspect
from contextlib import contextmanager

import fastcore.all as FC


In [ ]:
import dataclasses

from fastcore.test import *


----


# config

In [ ]:
#| export

class Config:
    def __repr__(self): return str(self.as_dict())
    def update(self, **kwargs):
        ks = set(FC.flatten(map(inspect.get_annotations, type(self).mro())))
        for k,v in kwargs.items():
            if k in ks:
                try: setattr(self, k, v)
                except AttributeError: pass
        return self
    def as_dict(self): return {k:getattr(self, k) for k in 
        FC.flatten(map(inspect.get_annotations, type(self).mro())) if hasattr(self, k)}
    @contextmanager
    def __call__(self, **kwargs):
        vv = vars(self).copy()
        old = {k:vv[k] for k in kwargs if k in vv}
        for k,v in kwargs.items(): setattr(self, k, v)
        yield
        for k in [k for k in vars(self).keys() if k not in vv]: delattr(self, k)
        for k,v in old.items(): setattr(self, k, v)

In [ ]:
@dataclasses.dataclass
class _Dc(Config):
    a: int = 1
    b: str = '2'
    c: dict = dataclasses.field(default_factory=lambda: {'c1': 1})

t = _Dc()
print(t)

test_eq(t.a, 1)
with t(a=3): test_eq(t.a, 3)
test_eq(t.a, 1)

t.update(a=3, d=10)
test_eq(t.a, 3)
test_is(hasattr(t, 'd'), False)

t

_Dc(a=1, b='2', c={'c1': 1})


_Dc(a=3, b='2', c={'c1': 1})

In [ ]:
class _A(Config):
    a: int = 1
    b: str = '2'

class _B(_A):
    a: int = 7
    c: dict
    def __init__(self, *args, **kwargs): self.c = {'c1': 1}

b = _B()
print(b)

test_eq(b.a, 7)
with b(a=3): test_eq(b.a, 3)
test_eq(b.a, 7)

b.update(a=3, d=10)
test_eq(b.a, 3)
test_is(hasattr(b, 'd'), False)

b

{'a': 7, 'c': {'c1': 1}, 'b': '2'}


{'a': 3, 'c': {'c1': 1}, 'b': '2'}

# Colophon
----


In [ ]:
#|hide
#|eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [ ]:
#|hide
#|eval: false

if FC.IN_NOTEBOOK:
    nb_path = '15_config.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)